In [ ]:
import torch
import numpy as np
import pandas as pd 
import sklearn

from torch import nn
from torch.utils.data import \
    Dataset as DS, \
    DataLoader as DL
from pandas import read_csv
from ipdb import set_trace

from AlexNet import AlexNet

import utils


### load data

In [ ]:
data_type = "undersampled"
X_train, X_test, Y_train, Y_test = utils.load_standard_data(data_type=data_type)

### dataset

In [ ]:
class Fraud_Dataset(DS):
    def __init__(self, X: pd.DataFrame, y: pd.DataFrame):
        self.X = X.values.astype(np.float32)
        self.Y = y.values.astype(np.float32)
    
    def __len__(self):
        return len(self.Y)
    
    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]


### training configuration

In [ ]:
dropout = 0.5
lr = 0.0002
batch_size = 100
num_classes = 1
epochs = 100
cuda_ind = 2

### training

In [ ]:
train_ds = Fraud_Dataset(X_train, Y_train)
train_dl = DL(train_ds, batch_size=batch_size, shuffle=True)

AN = AlexNet(num_classes=num_classes, dropout=dropout)
AN_opt = torch.optim.Adam(AN.parameters(), lr=lr)
if torch.cuda.is_available():
    AN.cuda(cuda_ind)
    
sigmoid = nn.Sigmoid() # final activation
bce = nn.BCELoss() # binary cross-entropy 

    
for e in range(epochs):
    
    # loss data
    loss_df = pd.DataFrame({"loss": []})
    
    for i, (X, y) in enumerate(train_dl):

        if torch.cuda.is_available():
            X = X.cuda(cuda_ind)
            y = y.cuda(cuda_ind)
        X = X.unsqueeze(1) # add a channel dimension
        
        AN_opt.zero_grad()
        
        y_probs = AN(X)
        loss = bce(sigmoid(y_probs), y)
        loss.backward() 
        loss_df = loss_df.append({"loss": loss.item()}, ignore_index=True)
        
        AN_opt.step()
    if e % 5 == 0:
        torch.save(AN.state_dict(), f"{utils._data_pth_}/models/AN_{data_type}_epochs_{e}.pth")
        loss_df.to_csv(f"{utils._data_pth_}/models/AN_{data_type}_epochs_{e}_loss.csv")
